In [ ]:
import pandas as pd
import numpy as np
train_txt = '../../DataSets/oppo_data_ronud2_20181107/data_train.txt'
val_txt = '../../DataSets/oppo_data_ronud2_20181107/data_vali.txt'
test_txt = '../../DataSets/oppo_round2_test_B/oppo_round2_test_B.txt'
train_data = pd.read_table(train_txt,
        names= ['prefix','query_prediction','title','tag','label'], header= None, encoding='utf-8',quoting=3).astype(str)
val_data = pd.read_table(val_txt,
        names = ['prefix','query_prediction','title','tag','label'], header = None, encoding='utf-8',quoting=3).astype(str)
test_data = pd.read_table(test_txt,
        names = ['prefix','query_prediction','title','tag'], header = None, encoding='utf-8',quoting=3).astype(str)

test_data['label'] = -1
data = pd.concat([train_data, val_data,test_data], ignore_index=True)

data['prefix_title'] = data['prefix'].apply(lambda x: x + "_")
data['prefix_title'] = data['prefix_title'] + data['title']

data['prefix_title_tag'] = data['prefix_title'].apply(lambda x: x + "_")
data['prefix_title_tag'] = data['prefix_title_tag'] + data['tag']

data['prefix'] = data['prefix'].apply(lambda x: x.lower())
data['title'] = data['title'].apply(lambda x: x.lower())

data['label'] = data['label'].apply(lambda x: int(x))

print('data shape : ', data.shape)

len_train = train_data.shape[0]
len_val = val_data.shape[0]

train_data = data[:len_train]
val_data = data[len_train:len_train+len_val]
test_data = data[len_train+len_val:]

len_train = train_data.shape[0]
len_val = val_data.shape[0]
len_test = test_data.shape[0]

In [ ]:
print("len_train",len_train)
print("len_val",len_val)
print("len_test",len_test)

pred = pd.read_table('../pred_online_threshold_0.38_feat_278_0.3793351329248621_86779.csv',
        names = ['predicted_score','label'],header=0 , encoding='utf-8',sep=',').astype(str)

test_data['predicted_label'] = pred['label'].apply(lambda x : int(x)).values
test_data['predicted_score'] = pred['predicted_score'].apply(lambda x : float(x)).values

In [ ]:
train_val_data = pd.concat([train_data, val_data])
prefixsAndCount = train_val_data.groupby(['prefix_title_tag'], as_index=False)['prefix_title_tag'].agg({'cnt': 'count'})
prefix_title_tag = prefixsAndCount['prefix_title_tag'].tolist()

titlesAndCount = train_val_data.groupby(['title'], as_index=False)['title'].agg({'cnt': 'count'})
titles = titlesAndCount['title'].tolist()

prefixs = train_val_data.groupby(['prefix'], as_index=False)['prefix'].agg({'cnt': 'count'})['prefix'].tolist()

print('prefix in : ',test_data[test_data['prefix'].isin(prefixs)].shape[0])
print('prefix not in',test_data[~test_data['prefix'].isin(prefixs)].shape[0])

print('prefixs : ', len(prefixs))
print("len of prefix_title_tag:",len(prefix_title_tag))
print('titles : ', len(titles))

d = {}
for i,row in prefixsAndCount.iterrows():
    d[row['prefix_title_tag']] = row['cnt']

d_title = {}
for i,row in titlesAndCount.iterrows():
    d_title[row['title']] = row['cnt']

# val model 1
test_feat_in = test_data[test_data['prefix_title_tag'].isin(prefix_title_tag)]
# val model2
test_feat_not_in = test_data[~test_data['prefix_title_tag'].isin(prefix_title_tag)]

len_in = test_feat_in.shape[0]
len_not_in = test_feat_not_in.shape[0]
# print(len_in,len_not_in)
print("prefix_title_tag in train : " , len_in)
print("prefix_title_tag not in train : " , len_not_in)

# val model 1
test_title_in = test_data[test_data['title'].isin(titles)]
# val model2
test_title_not_in = test_data[~test_data['title'].isin(titles)]

len_title_in = test_title_in.shape[0]
len_title_not_in = test_title_not_in.shape[0]

print("len of test in titles : ", len_title_in)
print("len of test not in titles : ", len_title_not_in)

test_title_in_prefix_not_in = test_title_in[(~test_title_in['prefix'].isin(prefixs))]
print('len of test which is not in prefix and in title' ,test_title_in_prefix_not_in.shape[0])
print(test_title_not_in[~test_title_not_in['prefix'].isin(prefixs)].shape)

title_counts = []
count1 = 0
count2 = 0
for i,row in test_title_in_prefix_not_in.iterrows():
    title_counts.append(d_title[row['title']])
    if d_title[row['title']] == 1:
        count1 = count1 + 1
    if d_title[row['title']] <= 2:
        count2 = count2 + 1
print(count1,count2)
test_title_in_prefix_not_in['title_count'] = title_counts

prefix_counts = []
count1 = 0
count2 = 0
for i,row in test_feat_in.iterrows():
    prefix_counts.append(d[row['prefix_title_tag']])
    if d[row['prefix_title_tag']] == 1:
        count1 = count1 + 1
    if d[row['prefix_title_tag']] <= 2:
        count2 = count2 + 1
print(count1,count2)
test_feat_in['prefix_count'] = prefix_counts

test_feat_in_one = test_feat_in[test_feat_in['prefix_count'] == 1]
print('test which only appear one in train',test_feat_in_one.shape[0])

test_title_in_one = test_title_in_prefix_not_in[test_title_in_prefix_not_in['title_count'] <=100]
print('title which only appear one in train',test_title_in_one.shape[0])

train_label = []

prefix_title_tag_one = test_feat_in_one['prefix_title_tag'].values
print(len(prefix_title_tag_one))
train_data_one = train_val_data[train_val_data['prefix_title_tag'].isin(prefix_title_tag_one)]
print(train_data_one.shape)

title_one = test_title_in_one['title'].values
print(len(title_one))
train_data_one_title = train_val_data[train_val_data['title'].isin(title_one)]
print(train_data_one_title.shape)

In [ ]:
train_data_one['label'] = train_data_one['label'].apply(lambda x : int(x))
temp = train_data_one.groupby('prefix_title_tag', as_index=False)['label'].agg({'click': 'sum', 'count': 'count'})
temp['ctr'] = temp['click'] / (temp['count'])
train_data_one = pd.merge(train_data_one, temp, on='prefix_title_tag', how='left')

train_data_one_title['label'] = train_data_one_title['label'].apply(lambda x : int(x))
temp = train_data_one_title.groupby('title', as_index=False)['label'].agg({'click': 'sum', 'count': 'count'})
temp['ctr'] = temp['click'] / (temp['count'])
train_data_one_title = pd.merge(train_data_one_title, temp, on='title', how='left')

train_ctr_dict = {}
for i,row in train_data_one_title.iterrows():
    train_ctr_dict[row['title']] = row['ctr']

#prefix_title_tag到label的转换
train_label_dict = {}
for i,row in train_data_one.iterrows():
    if i % 1000 == 0:
        print(i)
    if row['ctr'] > 0.38:
        train_label_dict[row['prefix_title_tag']] = 1
    else:
        train_label_dict[row['prefix_title_tag']] = 0

labels = []
cnt = 0
cnt_dou = 0
cnt_cugou = 0
cn_tao = 0
cn_xigua = 0
cn_wanneng = 0
for i,row in test_data.iterrows():
    if i % 1000 == 0:
        print(i)
    if row['prefix_title_tag'] in prefix_title_tag_one:
        labels.append(train_label_dict[row['prefix_title_tag']])
        cnt = cnt + 1
#     elif row['title'] in title_one:
#         if row['predicted_label'] == 0:
#             if train_ctr_dict[row['title']] > 0.4 and row['predicted_score'] > 0.2:
#                 labels.append(1)
#             else:
#                 labels.append(0)
#         else:
#             if train_ctr_dict[row['title']] < 0.6 and row['predicted_score'] < 0.4:
#                 labels.append(0)
#             else:
#                 labels.append(1)
    else:
        labels.append(row['predicted_label'])
print(cnt)
print(cnt_dou)
print(cnt_cugou)
print(cn_tao)
print(cn_xigua)
print(cn_wanneng)
# print(labels[:10])
print(np.sum(test_data['predicted_label']!=labels))

In [ ]:
print(np.sum(labels))
print(cnt_cugou)

In [ ]:
submission = pd.DataFrame()    
submission['label'] = labels
submission['label'].to_csv(
                'result_replace_with_ctr_0.38_1203.csv', sep=',', index=False)